In [6]:
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import np_utils
import keras
from keras import losses, optimizers, metrics

In [7]:
class ProteinLigandDataGenerator(keras.utils.Sequence):
    def __init__(self, csv_path, batch_size, shuffle=True, dim=(24,24,24), n_channels=2):
        self.df = self.prepare_df(csv_path)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = None
        self.dim = dim
        self.n_channels = n_channels
        self.on_epoch_end()
    
    def prepare_df(self, csv_path):
        # oversampling happens here, 
        match_count = 1
        random_count = 4
        supplement = random_count - match_count
        
        df = pd.read_csv(csv_path)
        same_idx = df[df['pro_id'] == df['lig_id']]
        duplicates = [same_idx]*supplement
        return df.append(duplicates, ignore_index=True)
        
    
    def __len__(self):
        '''
        Batches per epoch
        '''
        return int(len(self.df.index)/self.batch_size)
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]
        X, y = self.generate_data(indexes)
        
        return X, y
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df.index))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def generate_data(self, indexes):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        
        sub_df = self.df.iloc[indexes]
        sub_df.reset_index(inplace=True)
        for row in sub_df.itertuples():
            idx = row[0]
            full_X = np.load(row.dests)
            X[idx,:,:,:,:] = full_X
            y[idx] = row.score
            
        return X, y

In [8]:
train_generator = ProteinLigandDataGenerator('./data/csv/train_lig_pro_pairs.csv', 12)
test_generator = ProteinLigandDataGenerator('./data/csv/test_lig_pro_pairs.csv', 12)

In [9]:
inputs = Input(shape=(24,24,24,2))
x = Conv3D(96, kernel_size=3, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Conv3D(128, kernel_size=3, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPool3D()(x)
x = Conv3D(196, kernel_size=3, activation='relu')(x)
x = BatchNormalization()(x)
x = Conv3D(256, kernel_size=3, activation='relu')(x)
x = BatchNormalization()(x)
x1 = GlobalAveragePooling3D()(x)
x2 = GlobalMaxPool3D()(x)
x = Concatenate()([x1,x2])
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=predictions)
model.compile(loss=losses.mean_squared_error, optimizer=optimizers.Adam(), metrics=[metrics.mse])

In [10]:

history = model.fit_generator(train_generator, epochs=2, use_multiprocessing=True, 
                              workers=4, verbose=1, validation_data=test_generator)

Epoch 1/2
1801/1801 [==============================] - 285s 158ms/step - loss: 0.0450 - mean_squared_error: 0.0450 - val_loss: 0.0273 - val_mean_squared_error: 0.0273
Epoch 2/2
1801/1801 [==============================] - 281s 156ms/step - loss: 0.0223 - mean_squared_error: 0.0223 - val_loss: 0.0202 - val_mean_squared_error: 0.0202


In [11]:
model.save('models/try_maxpool_w_adam.h5')